# Search Based Software Engineering
## Exercise 01 , Task5 -  Iterated Local Search on Traveling Salesman Problem
### Team: 
### 1. Sagar Nagaraj Simha - sagar.nagaraj.simha[at]uni-weimar.de
####     Matrikel Nummer - 120797, Degree - Master's Computer Science for Digital Media
### 2. Mohd Saif Khan - mohd.saif.khan[at]uni-weimar.de 
####     Matrikel Nummer - 120803, Degree - Master's Human Computer Interaction

In [28]:
#Imports
import cmath
import math
from itertools import permutations
import datetime

In [29]:
def distance(p1, p2):
    """
    Returns the Euclidean distance of two points in the Cartesian Plane.

    >>> distance([3,4],[0,0])
    5.0
    
    """
    return ((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2) ** 0.5

In [30]:
def total_distance(points):
    """
    Returns the length of the path passing throught
    all the points in the given order.

    >>> total_distance([[1,2],[4,6]])
    5.0
    >>> total_distance([[3,6],[7,6],[12,6]])
    9.0
    """
    return sum([distance(point, points[index + 1]) for index, point in enumerate(points[:-1])])

#### Adjacency Matrix

We also need to specify a "distance matrix" that we can use to keep track of distances between cities. To generate a distance matrix for a set of (x,y) coordinates we will use the following function:

In [31]:
def cartesian_matrix(coordinates):
    '''
    Creates a distance matrix for the city coords using straight line distances
    computed by the Euclidean distance of two points in the Cartesian Plane.
    '''
    matrix = {}
    for i, p1 in enumerate(coordinates):
        for j, p2 in enumerate(coordinates):
            matrix[i,j] = distance(p1,p2)
    return matrix

This function takes a list of (x,y) tuples and outputs a dictionary that contains the distance between any pair of cities:

#### Read City Coordinates from File

In [32]:
def read_coords(file_handle):
    coords = []
    for line in file_handle:
        x,y = line.strip().split(',')
        coords.append((float(x), float(y)))
    return coords

with open('city100.txt', 'r') as coord_file:
    coords = read_coords(coord_file)
matrix = cartesian_matrix(coords)

On real world problems it may be more complicated to generate a distance matrix - you might need to take a map and calculate the real distances between cities.

#### Compute the Total Distance

In [33]:
def tour_length(matrix, tour):
    """Sum up the total length of the tour based on the distance matrix"""
    result = 0
    num_cities = len(list(tour))
    for i in range(num_cities):
        j = (i+1) % num_cities
        city_i = tour[i]
        city_j = tour[j]
        result += matrix[city_i, city_j]
    return result

#### Implementing Tweak Operators

We will implement the two tweak operators as generator functions that will return all possible versions of a route that can be made in one step of the generator (in a random order).

Generators are iterators which can be only iterated once.
They generate values on the fly and do not store them in memory.
By using a generator function, we can get each possiblility and perhaps decide to not generate any more variations.
This saves the overhead of generating all combinations at once.

In [34]:
import random

def all_pairs(size, shuffle = random.shuffle):
    r1 = list(range(size))
    r2 = list(range(size))
    if shuffle:
        shuffle(r1)
        shuffle(r2)
    for i in r1:
        for j in r2:
            yield(i,j) # yield is an iterator function
            # for each call of the generator it returns the next value in yield

In [35]:
from copy import deepcopy

# Tweak 1
def swapped_cities(tour):
    """
    Generator to create all possible variations where two 
    cities have been swapped
    """
    ap = all_pairs(len(tour))
    for i,j in ap:
        if i < j:
            copy = deepcopy(tour)
            copy[i], copy[j] = tour[j], tour[i]
            yield copy

# Tweak 2
def reversed_sections(tour):
    """
    Generator to return all possible variations where the
    section between two cities are swapped.
    It preserves entire sections of a route,
    yet still affects the ordering of multiple cities in one go.
    """
    ap = all_pairs(len(tour))
    for i,j in ap:
        if i != j:
            #print("indices from:",i, "to", j)
            copy = deepcopy(tour)
            if i < j:
                copy[i:j+1] = reversed(tour[i:j+1])
            else:
                copy[i+1:] = reversed(tour[:j])
                copy[:j] = reversed(tour[i+1:])
            if copy != tour: # not returning same tour
                yield copy
# usage
print("start tour swap:",[1,2,3,4])
for tour in swapped_cities([1,2,3,4]):
    print(tour)
print()
print("start tour reverse section:",[1,2,3,4])
for tour in reversed_sections([1,2,3,4]):
    print(tour)

start tour swap: [1, 2, 3, 4]
[2, 1, 3, 4]
[3, 2, 1, 4]
[4, 2, 3, 1]
[1, 2, 4, 3]
[1, 3, 2, 4]
[1, 4, 3, 2]

start tour reverse section: [1, 2, 3, 4]
[1, 3, 2, 4]
[4, 3, 1, 2]
[1, 4, 3, 2]
[3, 4, 2, 1]
[2, 3, 4, 1]
[3, 2, 1, 4]
[2, 1, 3, 4]
[4, 3, 2, 1]
[4, 1, 2, 3]
[4, 2, 3, 1]
[1, 2, 4, 3]


In [36]:
# Reverse city sections - Another Tweak Operator

In [37]:
import random
import numpy as np
import copy

def rev_city(tour):
    new_tour = copy.deepcopy(tour)
    i = random.randint(0,len(tour))
    j = random.randint(0,len(tour))
    if(i!=j):
        new_tour[min(i,j) : max(i,j)] = tour[min(i,j) : max(i,j)][::-1]
    return new_tour

In [38]:
#Example
tour = [i for i in range(20)]
print(tour)
print(tour[::-1])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


In [39]:
#Example
test = rev_city(tour)
print(test)

[7, 6, 5, 4, 3, 2, 1, 0, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


In [40]:
def init_random_tour(tour_length):
    tour = list(range(tour_length))
    random.shuffle(list(tour))
    return tour

init_function = lambda: init_random_tour(len(coords))
objective_function = lambda tour: tour_length(matrix, tour)

In [41]:
# n(n+1)/2 = timefactor. We need to find n of the series(1+2+3+...+n) = timefactor
# Solving the quadratic equation ax**2 + bx + c = 0
# n**2+n - 2*timefactor = 0
    
def SolutionToQuadraticEquation(timefactor):
    a=1
    b=1
    c=-2*timefactor
    
    # calculate the discriminant
    d = (b**2) - (4*a*c)

    # find two solutions
    sol1 = (-b-cmath.sqrt(d))/(2*a)
    sol1_r = sol1.real
    sol2 = (-b+cmath.sqrt(d))/(2*a)
    sol2_r = sol2.real

    if ((sol1_r>0) | (sol1_r==0)):
        return (math.ceil(sol1_r))
    elif ((sol2_r>0) | (sol2_r==0)):
        return (math.ceil(sol2_r))

In [42]:
#Example
SolutionToQuadraticEquation(28) # 28 (Timefactor) is the sum of n numbers in 1+2+3+4+...+n. The result is n

7

In [43]:
# The function returns a list of random time intervals/slots all amounting close to the max evaluations

def TimeSlots(min_eval_per_timeslot, desired_max_evaluations):
    T=[]
    timefactor = int(desired_max_evaluations/min_eval_per_timeslot) 
    
    # We need to find number of slots whose multiple with timefactor gives number of evaluations in that slot, so that the total time is around Max_Evaluations
    # number of slots is n in [1+2+3+...+n] = n(n+1)/2 = timefactor 
    number_of_slots = SolutionToQuadraticEquation(timefactor) 
    #Create timeslots with min eval per cycle
    for i in range(number_of_slots):
        T.append((i+1)*min_eval_per_timeslot)
    random.shuffle(T)
    return T, sum(T)

In [44]:
# Example
min_eval_per_timeslot = 100
desired_max_evaluations = 500000
timeslots_, Totalevals_apprxTimeslots = TimeSlots(min_eval_per_timeslot,desired_max_evaluations)
print("The time slots are")
print(timeslots_)
print("The function 'TimeSlots' will return a list of random time intervals where each is a multiple of min_eval_per_timeslot. The Total Number of evaluations amount to Totalevals_apprxTimeslots(this value would be slightly larger than desired_max_evaluations due to rounding off in the stage of finding 'n' in Quadratic equation)")
print(Totalevals_apprxTimeslots)

The time slots are
[1700, 5100, 7200, 3300, 4900, 8000, 3800, 8800, 2300, 7600, 4400, 2600, 6400, 1500, 3900, 5900, 4100, 4800, 1100, 6900, 8100, 100, 5000, 4500, 6100, 2500, 4600, 1400, 5500, 4300, 9200, 7100, 7700, 3000, 3500, 10000, 6800, 700, 5600, 8200, 2900, 1800, 9300, 7800, 4200, 8700, 5200, 800, 8400, 3100, 500, 2800, 8500, 2000, 3200, 7900, 3700, 2400, 4700, 1000, 9900, 2100, 1600, 9600, 1200, 4000, 1300, 5800, 8300, 7500, 7000, 5400, 400, 9800, 1900, 200, 9000, 2700, 6300, 300, 6500, 8600, 6200, 900, 6700, 3600, 9500, 3400, 9100, 2200, 9400, 7400, 5700, 8900, 6600, 5300, 600, 6000, 7300, 9700]
The function 'TimeSlots' will return a list of random time intervals where each is a multiple of min_eval_per_timeslot. The Total Number of evaluations amount to Totalevals_apprxTimeslots(this value would be slightly larger than desired_max_evaluations due to rounding off in the stage of finding 'n' in Quadratic equation)
505000


In [45]:
#NewHomeBase

#The NewHomeBase should lie somewhere between the extremes of RandomWalk and Hill Climbing when considering among local optima

def NewHomeBase(h,h_score,s,s_score):    
    if(random.random() < 0.5): #[0,1]
        if(s_score <= h_score):
            return s, s_score
        else:
            return h, h_score
    else:
        return s, s_score

In [46]:
#Perturb
# Here, Perturb is chosen to be two successive tweaks(of rev_city). The heuristic in choosing this is that two successive tweaks
# would probably provide a balance to escape the local hill, only onto the neighbouring hill

#"The goal of the Perturb function is to make a very large Tweak, big enough to likely escape the
#current local optimum, but not so large as to be essentially a randomization" - Essentials of Metaheuristics

def perturb(tour):
    tour = move_operator(tour)
    tour = move_operator(tour)
    #tour = init_function() # Poor results using this. may be because it's to the random extreme
    tour_score = objective_function(tour)
    return tour, tour_score

## Iterated Local Search

In [68]:
def ILS(init_function, move_operator, objective_function, min_eval_per_timeslot, desired_max_evaluations):
    '''
    Iterated Local Search
    '''
    # The function genertes Time slots, 
    #Totalevals_roundupTimeslots will be slightly > desired_max_evaluations since,
    #we round up each slot to the higher decimal in Solution to Quadratic Equation phase
    
    timeslots_, Totalevals_roundupTimeslots = TimeSlots(min_eval_per_timeslot,desired_max_evaluations)
    
    
    s = init_function()
    s_score = objective_function(s)
    
    h = s
    h_score = s_score
    
    best = s
    best_score = s_score
    
    #timeslots_ is a list of random time slots (where each slot is of varying duration) and the total slots amounts to maxevaluations.
    for timeslot in timeslots_ :
        
        for i in range(timeslot):
            move_made = False
            
            r = move_operator(s)
            r_score = objective_function(r)
    
            if r_score < s_score:
                s = r
                s_score = r_score
                move_made = True
        
        if s_score < best_score:
            best = s
            best_score = s_score
            print("BestScore",best_score)
            move_made = True
        
        h,h_score = NewHomeBase(h,h_score,s,s_score)
        s, s_score = perturb(h) 
                
    return (Totalevals_roundupTimeslots, best_score, best)

In [48]:
from PIL import Image, ImageDraw, ImageFont

def write_tour_to_img(coords, tour, title, img_file):
    padding = 20
    # shift all coords in a bit
    coords = [(x+padding,y+padding) for (x,y) in coords]
    maxx, maxy = 0,0
    for x,y in coords:
        maxx = max(x,maxx)
        maxy = max(y,maxy)
    maxx += padding
    maxy += padding
    img = Image.new("RGB",(int(maxx), int(maxy)), color=(255,255,255))
    
    font=ImageFont.load_default()
    d=ImageDraw.Draw(img);
    num_cities = len(tour)
    for i in range(num_cities):
        j = (i+1) % num_cities
        city_i = tour[i]
        city_j = tour[j]
        x1,y1 = coords[city_i]
        x2,y2 = coords[city_j]
        d.line((int(x1), int(y1), int(x2), int(y2)), fill=(0,0,0))
        d.text((int(x1)+7, int(y1)-5), str(i), font=font, fill=(32,32,32))
    
    
    for x,y in coords:
        x,y = int(x), int(y)
        d.ellipse((x-5, y-5, x+5, y+5), outline=(0,0,0), fill=(196,196,196))
    
    d.text((1,1), title, font=font, fill=(0,0,0))
    
    del d
    img.save(img_file, "PNG")

In [49]:
def reload_image_for_jupyter(filename):
    # pick a random integer with 1 in 2 billion chance of getting the same
    # integer twice
    import random
    __counter__ = random.randint(0,2e9)

    # now use IPython's rich display to display the html image with the
    # new argument
    from IPython.display import HTML, display
    display(HTML('<img src="./'+filename+'?%d" alt="Schema of adaptive filter" height="100">' % __counter__))


In [50]:
def do_ILS_evaluations(desired_max_evaluations , min_eval_per_timeslot, move_operator = rev_city):
    then = datetime.datetime.now()
    num_evaluations, best_score, best = ILS(init_function, move_operator, objective_function, min_eval_per_timeslot, desired_max_evaluations)
    now = datetime.datetime.now()

    print("computation time ", now - then)
    print("best score:", best_score)
    print("best route:", best)
    filename = "testILS"+str(desired_max_evaluations)+".PNG"
    write_tour_to_img(coords, best, filename, open(filename, "ab"))
    reload_image_for_jupyter(filename)

In [51]:
print("Check")

Check


In [63]:
move_operator = rev_city
min_eval_per_timeslot = 1000
desired_max_evaluations = 500000 #1000000
do_ILS_evaluations(desired_max_evaluations, min_eval_per_timeslot, move_operator)

# Avg score is around 4050 and takes ~ 50s for 500000 evaluations

BestScore 9878.970795164412
BestScore 4234.786106486358
BestScore 4209.606017206688
BestScore 4129.691682559481
BestScore 4092.537843207863
BestScore 4091.329886168242
BestScore 3941.5520451818334
computation time  0:00:50.403294
best score: 3941.5520451818334
best route: [28, 4, 2, 12, 93, 6, 45, 37, 58, 72, 95, 76, 55, 94, 32, 15, 53, 71, 47, 52, 11, 13, 64, 62, 75, 80, 85, 83, 49, 74, 8, 14, 43, 38, 23, 63, 41, 61, 26, 78, 1, 90, 97, 51, 48, 65, 84, 3, 54, 16, 36, 18, 22, 67, 88, 34, 21, 10, 96, 19, 31, 79, 20, 99, 50, 30, 9, 73, 86, 56, 39, 35, 0, 68, 5, 92, 66, 44, 77, 69, 60, 87, 82, 59, 57, 7, 81, 33, 46, 70, 24, 89, 29, 17, 27, 91, 40, 98, 25, 42]
